In [1]:
import requests # Para la conexiones con las paginas
from bs4 import BeautifulSoup as b # para extraer datos de archivos HTML y XML, leugo de la conexión con la página
import pandas as pd # Para guardar los datos en DataFrame
import re # Para utilizar expresiones regulares

In [2]:
def conexion(url):
    respueta = requests.get(url) # La función devuelve un objeto Response (una respuesta)
    conect = respueta.content # Si la respuesta fue positiva, entonces se procede a hacer la conexión
    soup = b(conect, 'html') # Se visualiza mejor la página para extraer texto específico
    return soup

Determinar el número de páginas de la consulta

In [3]:
url = 'https://siem.economia.gob.mx/establecimientos-publicos-x-criterios?id=&catEntidadFederativaFk=0&catActividad=0&catCamaraFk=~Jf3KrDTdu63&nombreComercial=&importa=2&exporta=2&publico=2&catEdoEstablecimientoFk=0&pageNum=0'

In [5]:
conection = conexion(url)
pagina0 = str(conection)
NumeroPaginas = int(re.findall(r'"pages":(\d+)', pagina0)[0]) #Extraemos el número de páginas con la expresión regular r'"pages":(\d+)', donde se muestra lo que se encuentre despúes de "pages": y que coincida con números (d).

In [6]:
NumeroPaginas

334

Para obtener los ID y los nombres de las empresas

In [79]:
ID = []
tradename = []

In [80]:
for i in range(1,NumeroPaginas+1):
    url = f'https://siem.economia.gob.mx/establecimientos-publicos-x-criterios?id=&catEntidadFederativaFk=0&catActividad=0&catCamaraFk=~Jf3KrDTdu63&nombreComercial=&importa=2&exporta=2&publico=2&catEdoEstablecimientoFk=0&pageNum={i}&orderBy=&desc=0'
    consulta = str(conexion(url))
    ID.extend(re.findall(r'"id":"(\w+)', consulta)) # esta expresión regular nos devuelve los caracteres alfa numéricos que se encuentren despues de '"id":'
    tradename.extend(re.findall(r'"nombreComercial":"([\w\s|,.]+)', consulta)) # Como los nombres comerciales de las empresas pueden contener caracteres alfa numércios (w), espacios en blanco (s) comas, puntos y otro caracter especial (,.|), entonces se debe el conjunto [\w\s|,.].

In [81]:
len(ID) # Se verifica que se haya obtenido los ID de las 3333 empresas

3333

In [82]:
len(tradename) # Se verifica que se haya obtenido los nombres comerciales de las 3333 empresas

3333

Para conseguir número de celular y correo

In [83]:
telefono = []
correo = []

In [ ]:
for i in ID:
    url = f'https://siem.economia.gob.mx/establecimiento-ubicacion.json?id={i}'
    consulta = requests.get(url).json()
    telefono.append(consulta["telefono"]) # Se trabaja como una consulta al .json que nos devuelve la url
    correo.append(consulta["mail"]) # Se trabaja como una consulta al .json que nos devuelve la url

In [125]:
len(telefono)

3333

In [127]:
len(correo)

3333

Creando la tabla de los datos extraidos (DataFrame)

In [128]:
Datos = pd.DataFrame( {'ID': ID, 'TRADENAME': tradename, 'TELEFONO' : telefono, 'CORREO': correo})

In [129]:
Datos

,ID,TRADENAME,TELEFONO,CORREO
0,274930,"Spat Mail, S.A. de C.V.",6141802815,JURIDICO@SPATMAIL.COM
1,1167188,"Adhesivos del Bajío, S. de R.L. de C.V.",4777150105,gerencialeon@ade1000.com
2,1237133,"SPAT MAILSpat Mail, S.A. de C.V.",6141802815,JURIDICO@SPATMAIL.COM
3,2779742,Mauricio Alfonso Alvarez Hidalgo,7773124100,FACTURACIONC@HOTMAIL.COM
4,4032155,Horacio Hurtado Hernandez,8682110292,lachoexpress@prodigy.net.mx
...,...,...,...,...
3328,zs6h6a6bsi,"Togo Delivery, S.A. de C.V.",5542021691,msanchez@ferac.com.mx
3329,zttzkcg180,"Transportes de Hidrocarburos, S.A. de C.V.",5557695716,None
3330,zu61ul1pv4,"Express J.L. Nava, S.A. de C.V.",8262610771,None
3331,zuz7g0cbow,Juan Pedro Castañeda Palos,4492314839,NATHALIE.NAVARRO@TCHHIDROEXPRESS.COM


Guardamos la tabla en un .csv

In [131]:
Datos.to_csv('EmpresasDeLogísticaMéxico.csv', index= False, encoding = '')